In [75]:
#Importing read-sm-files.py
import ReadSMFIles
from Resource import Resource
from Task import Task
import random
from typing import List, Tuple

#reading j30.sm/j301_1.sm file
sm_file = ReadSMFIles.SMFileParser.parse_sm_file("j30.sm/j301_1.sm")

print(sm_file[2].successors)


0        [2, 3, 4]
1      [6, 11, 15]
2       [7, 8, 13]
3       [5, 9, 10]
4             [20]
5             [30]
6             [27]
7     [12, 19, 27]
8             [14]
9         [16, 25]
10        [20, 26]
11            [14]
12        [17, 18]
13            [17]
14            [25]
15        [21, 22]
16            [22]
17        [20, 22]
18        [24, 29]
19        [23, 25]
20            [28]
21            [23]
22            [24]
23            [30]
24            [30]
25            [31]
26            [28]
27            [31]
28            [32]
29            [32]
30            [32]
31              []
Name: successors, dtype: object


In [72]:
#Creating resources
r1 = int(sm_file[4].R1[0])
r2 = int(sm_file[4].R2[0])
r3 = int(sm_file[4].R3[0])
r4 = int(sm_file[4].R4[0])

R1 = Resource('R1', r1)
R2 = Resource('R2', r2)
R3 = Resource('R3', r3)
R4 = Resource('R4', r4)

resources = [R1, R2, R3, R4]


In [73]:
#Creating jobs
jobs_enumerate = sm_file[3].jobnr
jobs_duration = sm_file[3].duration
jobs_resources = sm_file[3].resources
jobs_successors = sm_file[2].successors

jobs = [None for _ in jobs_enumerate]


for i in jobs_enumerate:
    jobs[i - 1] = Task(str(i), jobs_duration[i - 1])

for i in range(len(resources)):
    for j in range(len(jobs)):
        jobs[j].add_renewable_resource(resources[i], jobs_resources[j][i])

for i in range(len(jobs)):
    successors = jobs_successors[i]
    for j in successors:
        jobs[i].add_sucessor(jobs[j - 1])
    
for job in jobs:
    print([sucessor.name for sucessor in job.predecessors])

    

    



[]
['1']
['1']
['1']
['4']
['2']
['3']
['3']
['4']
['4']
['2']
['8']
['3']
['9', '12']
['2']
['10']
['13', '14']
['13']
['8']
['5', '11', '18']
['16']
['16', '17', '18']
['20', '22']
['19', '23']
['10', '15', '20']
['11']
['7', '8']
['21', '27']
['19']
['6', '24', '25']
['26', '28']
['29', '30', '31']



Genetic Algorithm



In [74]:
#Genetic Algorithm

#Creating random population
def create_population(population_size, jobs):
    population = []
    for i in range(population_size):
        population.append(jobs.copy())
        random.shuffle(population[i])
    return population

population = create_population(10, jobs)

for i in range(len(population)):
    print([job.name for job in population[i]])
    print("\n")

['18', '26', '22', '20', '1', '11', '19', '30', '32', '14', '31', '12', '29', '21', '25', '17', '15', '13', '23', '24', '16', '3', '4', '27', '5', '6', '7', '28', '9', '8', '2', '10']


['31', '9', '28', '22', '30', '27', '6', '13', '2', '7', '19', '21', '17', '11', '5', '3', '20', '8', '12', '18', '29', '26', '10', '4', '16', '23', '14', '24', '32', '15', '1', '25']


['28', '12', '10', '26', '30', '1', '13', '24', '25', '11', '3', '20', '19', '16', '2', '21', '27', '18', '15', '23', '8', '17', '6', '9', '31', '29', '22', '4', '32', '7', '14', '5']


['26', '11', '25', '13', '6', '23', '8', '19', '7', '20', '10', '15', '17', '29', '9', '2', '27', '3', '22', '30', '18', '5', '4', '32', '14', '12', '31', '24', '16', '28', '21', '1']


['32', '20', '3', '11', '6', '15', '5', '1', '10', '23', '7', '14', '12', '27', '19', '25', '28', '18', '24', '22', '8', '16', '21', '13', '29', '17', '9', '4', '2', '30', '31', '26']


['19', '10', '11', '29', '32', '17', '9', '1', '21', '30', '20', '7', 

Crossover

In [87]:

class OnePointCrossoverOperator:
    def __init__(self, crossover_rate: float = 0.8):
        self.crossover_rate = crossover_rate

    def __call__(
            self, parent1: List[Task], parent2: List[Task]
    ) -> Tuple[List[Task], List[Task]]:
        if random.random() > self.crossover_rate:
            return parent1, parent2

        crossover_point = random.randint(0, len(parent1) - 1)
        child1 = parent1[:crossover_point]
        for value in parent2:
            if value not in child1:
                child1.append(value)
        
        child2 = parent2[:crossover_point]
        for value in parent1:
            if value not in child2:
                child2.append(value)

        return child1, child2


crossover = OnePointCrossoverOperator()

child1, child2 = crossover(population[0], population[1])

print([job.name for job in population[0]])
print("\n")
print([job.name for job in population[1]])
print("\n")
print([job.name for job in child1])
print("\n")
print([job.name for job in child2])


['18', '26', '22', '20', '1', '11', '19', '30', '32', '14', '31', '12', '29', '21', '25', '17', '15', '13', '23', '24', '16', '3', '4', '27', '5', '6', '7', '28', '9', '8', '2', '10']


['31', '9', '28', '22', '30', '27', '6', '13', '2', '7', '19', '21', '17', '11', '5', '3', '20', '8', '12', '18', '29', '26', '10', '4', '16', '23', '14', '24', '32', '15', '1', '25']


['18', '26', '22', '20', '1', '11', '19', '30', '32', '14', '31', '9', '28', '27', '6', '13', '2', '7', '21', '17', '5', '3', '8', '12', '29', '10', '4', '16', '23', '24', '15', '25']


['31', '9', '28', '22', '30', '27', '6', '13', '2', '7', '19', '18', '26', '20', '1', '11', '32', '14', '12', '29', '21', '25', '17', '15', '23', '24', '16', '3', '4', '5', '8', '10']
